# Container Benchmark Analysis

Interactive performance analysis comparing container implementations (vectors, sets, and maps).

**Click legend items to show/hide series. Hover for details.**

In [ ]:
import sys
from pathlib import Path

import polars as pl
import plotly.express as px

sys.path.insert(0, str(Path.cwd().parent.parent / "src"))
from benchmarks import BuildType, load_benchmark

# Vectors

In [ ]:
repo_path = Path("../../..")
vector_df = load_benchmark(repo_path, "containers_vector")

if vector_df.is_empty():
    raise RuntimeError("No vector results found. Run: cd scripts && uv run bench run")

vector_df = vector_df.with_columns(
    pl.col("name").str.extract(r"<(.+)>/", 1).alias("vector_impl"),
    (pl.col("name").str.extract(r"<(.+)>/", 1) + " (" + pl.col("compiler") + ")").alias("series")
)

vector_df = vector_df.filter(pl.col("build_type") == BuildType.RELEASE.value)

## Vector: Mixed Operations (Push + Iterate)

In [ ]:
test_df = vector_df.filter(
    (pl.col("test_name") == "mixed") & (pl.col("label") == "pushes+iters")
).sort(["series", "size"])

if not test_df.is_empty() and "items_per_second_M" in test_df.columns:
    fig = px.line(
        test_df,
        x="size",
        y="items_per_second_M",
        color="series",
        markers=True,
        log_x=True,
        title="Vector Mixed Operations (Push + Iterate)",
        labels={"size": "Vector Size", "items_per_second_M": "Throughput (M/s)", "series": "Implementation"},
        height=600
    )
    fig.update_traces(mode="lines+markers")
    fig.show()

## Vector: Mutable Iteration

In [ ]:
test_df = vector_df.filter(
    (pl.col("test_name") == "iter_mut") & (pl.col("label") == "mutations")
).sort(["series", "size"])

if not test_df.is_empty() and "items_per_second_M" in test_df.columns:
    fig = px.line(
        test_df,
        x="size",
        y="items_per_second_M",
        color="series",
        markers=True,
        log_x=True,
        title="Vector Mutable Iteration",
        labels={"size": "Vector Size", "items_per_second_M": "Throughput (M/s)", "series": "Implementation"},
        height=600
    )
    fig.update_traces(mode="lines+markers")
    fig.show()

# Sets

In [ ]:
set_df = load_benchmark(repo_path, "containers_set")

if set_df.is_empty():
    raise RuntimeError("No set results found. Run: cd scripts && uv run bench run")

set_df = set_df.with_columns(
    (pl.col("impl") + " [" + pl.col("item_size").cast(pl.Utf8) + "B] (" + pl.col("compiler") + ")").alias("series")
)

set_df = set_df.filter(pl.col("build_type") == BuildType.RELEASE.value)

## Set: Iterator Performance (uint32_t)

In [ ]:
test_df = set_df.filter(
    (pl.col("test_name") == "iterator") & (pl.col("item_size") == 4)
).sort(["series", "size"])

if not test_df.is_empty() and "items_per_second_M" in test_df.columns:
    fig = px.line(
        test_df,
        x="size",
        y="items_per_second_M",
        color="series",
        markers=True,
        log_x=True,
        title="Set Iterator (uint32_t)",
        labels={"size": "Set Size", "items_per_second_M": "Throughput (M/s)", "series": "Implementation"},
        height=600
    )
    fig.update_traces(mode="lines+markers")
    fig.show()

## Set: Mixed Operations (size_t)

In [ ]:
test_df = set_df.filter(
    (pl.col("test_name") == "mixed_sequential") & (pl.col("item_size") == 8)
).sort(["series", "size"])

if not test_df.is_empty() and "items_per_second_M" in test_df.columns:
    fig = px.line(
        test_df,
        x="size",
        y="items_per_second_M",
        color="series",
        markers=True,
        log_x=True,
        title="Set Mixed Operations (size_t)",
        labels={"size": "Set Size", "items_per_second_M": "Throughput (M/s)", "series": "Implementation"},
        height=600
    )
    fig.update_traces(mode="lines+markers")
    fig.show()

# Maps

In [ ]:
map_df = load_benchmark(repo_path, "containers_map")

if map_df.is_empty():
    raise RuntimeError("No map results found. Run: cd scripts && uv run bench run")

map_df = map_df.with_columns(
    (pl.col("impl") + " [" + pl.col("key_size").cast(pl.Utf8) + "→" + pl.col("value_size").cast(pl.Utf8) + "B] (" + pl.col("compiler") + ")").alias("series")
)

map_df = map_df.filter(pl.col("build_type") == BuildType.RELEASE.value)

## Map: Iteration Performance (uint32_t → uint32_t)

In [ ]:
test_df = map_df.filter(
    (pl.col("test_name") == "iterate")
    & (pl.col("key_size") == 4)
    & (pl.col("value_size") == 4)
).sort(["series", "size"])

if not test_df.is_empty() and "items_per_second_M" in test_df.columns:
    fig = px.line(
        test_df,
        x="size",
        y="items_per_second_M",
        color="series",
        markers=True,
        log_x=True,
        title="Map Iteration (uint32_t → uint32_t)",
        labels={"size": "Map Size", "items_per_second_M": "Throughput (M/s)", "series": "Implementation"},
        height=600
    )
    fig.update_traces(mode="lines+markers")
    fig.show()

## Map: derive-C Implementations Only

In [ ]:
test_df = map_df.filter(
    (pl.col("test_name") == "iterate")
    & (pl.col("key_size") == 4)
    & (pl.col("value_size") == 4)
    & pl.col("impl").str.starts_with("derive-c/")
).sort(["series", "size"])

if not test_df.is_empty() and "items_per_second_M" in test_df.columns:
    fig = px.line(
        test_df,
        x="size",
        y="items_per_second_M",
        color="series",
        markers=True,
        log_x=True,
        title="derive-C Maps (uint32_t → uint32_t)",
        labels={"size": "Map Size", "items_per_second_M": "Throughput (M/s)", "series": "Implementation"},
        height=600
    )
    fig.update_traces(mode="lines+markers")
    fig.show()